In [2]:
import numpy as np
import pandas as pd

# Decision Tree

In [75]:
X = np.array([
    [1, 5, 2],
    [2, 3, 1],
    [3, 6, 2],
    [2, 7, 3],
    [3, 2, 2],
    [6, 4, 3],
    [7, 5, 4],
    [8, 3, 4],
    [7, 8, 5],
    [9, 6, 5]
])

y = np.array([0,0,0,0,0, 1,1,1,1,1])

In [76]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):

        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        

In [77]:
def gini_impurity(y):
    
    y = np.array(y)
    _, counts = np.unique(y, return_counts = True)
    p = counts/np.sum(counts)
    gini = 1 - (np.sum(p**2))

    return gini

In [78]:
def best_split(X, y):
    
    G_parent = gini_impurity(y)
    best_gain = 0
    best_feature = None

    #Iterujemy po cechach
    for l in range(len(X.T)):
        potential_threshold = np.unique(X[:, l])

        for t in potential_threshold:
            mask_left = X[:, l] <= t
            mask_right = X[:, l] > t
            
            left_gini = gini_impurity(y[mask_left])
            right_gini = gini_impurity(y[mask_right])
            
            n_left = len(y[X[:, l] <= t])
            n_right = len(y[X[:, l] > t])
            
            weighted_gini = n_left/len(y) * left_gini + n_right/len(y) * right_gini
            gain = G_parent - weighted_gini
    
            if gain > best_gain:
                best_gain = gain
                best_threshold = t
                best_feature = l
                left_idx = np.arange(len(y))[mask_left]
                right_idx = np.arange(len(y))[mask_right]

        if best_feature is None:
            return None, None, None, None
            
    return best_feature, best_threshold, left_idx, right_idx
            
    

In [85]:
def buildtree(X, y, depth, max_depth):
    
    if depth >= max_depth:
        return Node(value = np.bincount(y).argmax())

    else:
        
        best_feature, best_threshold, left_idx, right_idx = best_split_RF(X, y)
        node = Node(feature_index = best_feature, threshold = best_threshold)
        
        if best_feature is None:
            return Node(value = np.bincount(y).argmax())
        
        X_left, X_right = X[left_idx], X[right_idx]
        y_left, y_right = y[left_idx], y[right_idx]
                    
        node.left = buildtree(X_left, y_left, depth + 1, max_depth)
        node.right = buildtree(X_right, y_right, depth + 1, max_depth)

    
    return node
        

In [86]:
def predict_one(X, node):
                
    # Jeżeli węzeł jest liściem zwracamy value          
    if node.value is not None:
        return node.value

    
    if X[node.feature_index] <= node.threshold:
        return predict_one(X, node.left)
    
    else:
        return predict_one(X, node.right)
        

In [87]:
class DecisionTree:
    def __init__(self, max_depth = 4):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = buildtree(X, y, depth = 0, max_depth = self.max_depth)

    def predict(self, X):
        return np.array([predict_one(x, self.root) for x in X])

# Random Forest

In [88]:
def best_split_RF(X, y):
    
    G_parent = gini_impurity(y)
    best_gain = 0
    best_feature = None
    features = np.random.choice(X.shape[1], size = np.sqrt(X.shape[1]).astype(int), replace=False)

    #Iterujemy po wybranych cechach
    for f in features:
        potential_threshold = np.unique(X[:, f])

        for t in potential_threshold:
            mask_left = X[:, f] <= t
            mask_right = X[:, f] > t
            
            left_gini = gini_impurity(y[mask_left])
            right_gini = gini_impurity(y[mask_right])
            
            n_left = np.sum(mask_left)
            n_right = np.sum(mask_right)
            
            weighted_gini = n_left/len(y) * left_gini + n_right/len(y) * right_gini
            gain = G_parent - weighted_gini
    
            if gain > best_gain:
                best_gain = gain
                best_threshold = t
                best_feature = f
                left_idx = np.arange(len(y))[mask_left]
                right_idx = np.arange(len(y))[mask_right]

        if best_feature is None:
            return None, None, None, None
            
    return best_feature, best_threshold, left_idx, right_idx
            
    

In [91]:
class RandomForest:
    def __init__(self, n_trees=100, max_depth=None, max_features=np.sqrt(len(X.T)).astype(int)):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def random_samples(self, X, y):
        indices = np.random.choice(np.arange(len(X)), size = 6)
        X_set, y_set = X[indices], y[indices]

        return X_set, y_set
        

    def fit(self, X, y):
        for t in range(self.n_trees):
            X_set, y_set = self.random_samples(X, y)
            tree = DecisionTree()
            tree.fit(X_set, y_set)
            self.trees.append(tree)

    def one_tree_predict(self, x, tree):
                    
        # Jeżeli węzeł jest liściem zwracamy value          
        if tree.value is not None:
            return tree.value
    
        
        if x[tree.feature_index] <= tree.threshold:
            return self.one_tree_predict(x, tree.left)
        
        else:
            return self.one_tree_predict(x, tree.right)

        
    def predict_one(self, x):
        lista_glosow = []
    
        for tree in self.trees:
            lista_glosow.append(self.one_tree_predict(x, tree.root))
    
        return np.bincount(lista_glosow).argmax()

    
    def predict(self, X):
        return [self.predict_one(x) for x in X]
        

In [101]:
rf = RandomForest(n_trees = 5, max_depth = 4)
rf.fit(X, y)
y_pred = rf.predict(X)
print(f"Accuracy: {np.mean((y_pred == y))}")

Accuracy: 1.0
